# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Read dataframe with 16797607 records.
Dataframe has 144797 records between (datetime.datetime(2021, 1, 6, 0, 0), datetime.datetime(2021, 1, 6, 0, 0)).
Subsetted dataframe with 100000 records.
Record num : 100000
Wall time: 15.4 s


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)
        continue

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    data = get_trend_fwd(data)    
    data = get_position(data)
    
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  
    
    data = calculate_pl(data)
#     data = simple_take_profit(data)
    data = dynamic_take_profit(data)
    data = simple_stop_loss(data)
    data = make_order(data)

100%|█████████████████████████████████| 100000/100000 [11:07<00:00, 149.83it/s]


In [4]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 100000
Hours     : 15.16
Recs/hour : 6596
Recs/min  : 109
Recs/sec  : 1.8


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,position,to_order,h_line_angle,l_line_angle,...,stop_loss_pip,pl_move_trail_trigger,pl_move_min,h,l,long_open,long_close,short_open,short_close,pl
0,20210106 00:00:00.067,1.22985,1.22990,1.229875,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210106 00:00:00.169,1.22987,1.22992,1.229895,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210106 00:00:00.270,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210106 00:00:00.372,1.22989,1.22994,1.229915,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210106 00:00:00.980,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pl_list = data['df']['pl'][data['df']['pl'].notnull()]
print(pl_list)
print('-----------------------------')
print(pl_list.value_counts())
print('-----------------------------')
print(sum(pl_list))

20641    0.00064
33157    0.00096
36539    0.00081
44532    0.00101
49645   -0.00110
65264    0.00093
69529    0.00083
83375   -0.00088
94073   -0.00126
96755   -0.00122
Name: pl, dtype: float64
-----------------------------
 0.00093    1
 0.00064    1
-0.00126    1
-0.00110    1
-0.00088    1
 0.00101    1
-0.00122    1
 0.00081    1
 0.00096    1
 0.00083    1
Name: pl, dtype: int64
-----------------------------
0.0007200000000000002


In [6]:
plot_graph(data)

In [7]:
data['df'].to_csv('data/temp.csv')

PermissionError: [Errno 13] Permission denied: 'data/temp.csv'

In [ ]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)
send_telegram_message(f'Run Complete at : {sum(pl_list)}')